In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neural_network import MLPClassifier,MLPRegressor

from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
df=pd.read_csv('/content/forestfires.csv')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df['month'].unique()

In [ ]:
df['day'].unique()

In [ ]:
df.head(10)

**Preprocessing**

In [ ]:
def ordinal_encoding(df,column,ordering):
  df=df.copy()
  df[column]=df[column].apply(lambda x: ordering.index(x))
  return df

In [ ]:
def preprocessing(df,task):
  df=df.copy()
  df=ordinal_encoding(
      df,
      column='month',
      ordering=[
                 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep',
       'oct', 'nov', 'dec'
       ]
  )
  df=ordinal_encoding(
      df,
      column='day',
      ordering=['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
  )
  if task=='Regression':
    Y=df['area']
  elif task=='Classification':
    Y=df['area'].apply(lambda x: 1 if x>0 else 0)

  X=df.drop('area',axis=1)

  X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.65,shuffle=True,random_state=1)

  scaler=StandardScaler()
  scaler.fit(X_train)

  X_train=pd.DataFrame(scaler.transform(X_train),columns=X.columns)
  X_test=pd.DataFrame(scaler.transform(X_test),columns=X.columns)

  return X_train,X_test,Y_train,Y_test

**Splitting & Testing Models:**

In [ ]:
X_train,X_test,Y_train,Y_test=preprocessing(df,task='Regression')

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
Y_train.head()

In [ ]:
Y_test.head()

Linear **Regression**

In [ ]:
#The maximum val. of R^2 can be 1.0 that signifies that Linear Regression is working really well.
#Here R^2 score is 0.02051 which is really low signifying that LinearRegression is not doing well.
linear_reg_model=LinearRegression()
linear_reg_model.fit(X_train,Y_train)

print("Performance of Linear Regression R^2 metric {:.5f}".format(linear_reg_model.score(X_train,Y_train)))

**MLP Regressor Model**

In [ ]:
# We can see the MLP Regressor Model Works better than Linear Regression,however its works horrible as well.
mlp_reg_model=MLPRegressor().fit(X_train,Y_train)
print("Performance of MLP Regressor Model R^2 metric {:.5f}".format(mlp_reg_model.score(X_train,Y_train)))

**Decision Tree Regressor**

In [ ]:
#R2 score negative not a good fit!
reg = dtr(random_state = 42)
reg.fit(X_train, Y_train)
Y_pred = reg.predict(X_test)
print("MSE =", mse(Y_pred, Y_test))
print("MAE =", mae(Y_pred, Y_test))
print("R2 Score =", r2_score(Y_pred, Y_test))

Random Forest Regressor

In [ ]:
#This works even worse than Decision Tree Regressor
regr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
regr.fit(X_train, Y_train)
Y_pred = regr.predict(X_test)
print("MSE =", mse(Y_pred, Y_test))
print("MAE =", mae(Y_pred, Y_test))
print("R2 Score =", r2_score(Y_pred, Y_test))

**Logistic Regression**

In [ ]:
X_train,X_test,Y_train,Y_test=preprocessing(df,task='Classification')

In [ ]:
Y_train

In [ ]:
log_reg_model=LogisticRegression()
log_reg_model.fit(X_train,Y_train)

print('Logistic Regression Accuracy, {:.5f}%'.format(log_reg_model.score(X_test,Y_test)*100))

**Neural Network Classifier**

In [ ]:
nn_classifier_model=MLPClassifier(activation='relu',hidden_layer_sizes=(16,16),n_iter_no_change=100,solver='adam')
nn_classifier_model.fit(X_train,Y_train)